## ideas:

- cluster map
- heatmap, spending at hour on weekday

In [1]:
import pandas as pd
from sentence_transformers import SentenceTransformer
from sklearn.cluster import KMeans
from sklearn.manifold import TSNE
import umap
from src.dashboard_utility import get_all_bank_data
import plotly.express as px

import plotly.io as pio
pio.templates.default = "plotly"

/Users/hannesmoehring/.pyenv/versions/finance_overview/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-11-27 10:47:23.039 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2025-11-27 10:47:23.040 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager


In [2]:
comdirect_df, traderepublic_df, olb_df = get_all_bank_data()
df = comdirect_df.copy()

2025-11-27 10:47:23.080 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2025-11-27 10:47:23.080 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-27 10:47:23.153 
  command:

    streamlit run /Users/hannesmoehring/.pyenv/versions/finance_overview/lib/python3.13/site-packages/ipykernel_launcher.py [ARGUMENTS]
2025-11-27 10:47:23.153 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-27 10:47:23.153 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-27 10:47:23.154 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
/Users/hannesmoehring/Documents/Projekte/dev/finance_overview/src/parsers.py:38: UserWarning: Could not infer format, so each element 

In [3]:
df = df[df["amount"] < 0]
df["amount"] = df["amount"].abs()

model = SentenceTransformer("all-MiniLM-L6-v2")
embeddings = model.encode(df["details"].tolist(), normalize_embeddings=True)

In [4]:
kmeans = KMeans(n_clusters=10, random_state=9042003)
df["cluster"] = kmeans.fit_predict(embeddings)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [5]:
# reducer = umap.UMAP(
#     n_neighbors=8,      
#     min_dist=0.0,       
#     spread=1.0,
#     metric="cosine"     
# )
# df[["x", "y"]] = reducer.fit_transform(embeddings)

tsne = TSNE(
    n_components=2,
    perplexity=30,
    learning_rate=200,
    metric="cosine"
)

df[["x", "y"]] = tsne.fit_transform(embeddings)

agg = df.groupby(["details", "cluster"]).agg(
    total_amount=("amount", "sum"),
    x=("x", "mean"),
    y=("y", "mean")
).reset_index()

agg["cluster"] = agg["cluster"].astype("category")

In [6]:
fig = px.scatter(
    agg,
    x="x", y="y",
    size="total_amount",
    color="cluster",
    hover_name="details",
    size_max=60
)
# st.plotly_chart(fig, use_container_width=True)
fig.show()

In [7]:
print(agg["cluster"].nunique(), agg["cluster"].unique()[:10])

10 [0, 9, 2, 8, 4, 7, 6, 1, 3, 5]
Categories (10, int32): [0, 1, 2, 3, ..., 6, 7, 8, 9]


In [9]:
agg

,details,cluster,total_amount,x,y
0,3leafs by Idris,0,54.40,-38.840233,-217.145523
1,ALDI NORD Buchungstext:,9,9.22,239.124466,-114.862244
2,ALDI SUeD Buchungstext:,2,22.55,243.721649,-126.552734
3,AVIA TANKSTELLE Buchungstext:,9,102.80,74.182449,-74.214722
4,Adelholzener Bar Buchungstext:,9,5.95,141.504608,-47.132740
...,...,...,...,...,...
162,ZWEIRAD-CENTER STADLER Buchungstext:,8,531.24,141.779083,83.608513
163,davengo GmbH Buchungstext:,2,49.00,-102.881981,-117.006897
164,"dean&david Buchungstext: dean&david,",2,8.90,141.805573,-99.113197
165,netcup GmbH Buchungstext:,2,93.25,-118.555519,-139.100769


In [11]:
cluster_sum = agg.groupby("cluster", as_index=False)["total_amount"].sum()
cluster_sum

/var/folders/sg/6kgs67jd7x78hnddmpl4_x1c0000gn/T/ipykernel_24544/3923723460.py:1: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



,cluster,total_amount
0,0,1558.39
1,1,2009.98
2,2,2491.83
3,3,983.91
4,4,228.07
5,5,339.00
6,6,1633.51
7,7,581.33
8,8,3759.09
9,9,1874.23
